In [5]:
import numpy as np
import matplotlib.pyplot as plt

import ot.apdamd.ot
from ot.greenkhorn.ot import OT as greenkhorn
from ot.apdamd.ot import OT as apdamd
from ot.tests.sample_problem import sample_gaussian_OT_exact
from ot.ot import cost
import timeit

In [6]:
# Create the problem
n = 1  # dim
N = 100  # number of points
Gaussian1 = (np.zeros(n), np.eye(n))
Gaussian2 = (np.ones(n), np.eye(n) + np.triu(np.ones(n)) * 0.5)
my_problem = sample_gaussian_OT_exact(N, n, (Gaussian1, Gaussian2))


In [7]:
epss = np.linspace(1, 2, 10)
timings_greenkhorn = dict()
timings_apdamd = dict()
std_timings_greenkhorn = dict()
std_timings_apdamd = dict()
transport_plans_greenkhorn = dict()
transport_plans_apdamd = dict()
costs_greenkhorn = dict()
costs_apdamd = dict()
realised_iters_greenkhorn = dict()
realised_iters_apdamd = dict()
theoretical_iters_greenkhorn = dict()
theoretical_iters_apdamd = dict()
converge_greenkhorn = dict()
converge_apdamd = dict()

ENABLE_GREENKHORN = False

n_iter = None

# Theoretical bounds on hte number of iterations
for i, eps in enumerate(epss):
    theoretical_iters_apdamd[(n_iter, eps)] = ot.apdamd.ot.theoretical_bound_on_iter(*my_problem[1:], eps)
    theoretical_iters_greenkhorn[(n_iter, eps)] = ot.greenkhorn.ot.theoretical_bound_on_iter(*my_problem[1:], eps)


In [ ]:
for i, eps in enumerate(epss):
    if ENABLE_GREENKHORN:
        transport_plans_greenkhorn[(n_iter, eps)], _ = greenkhorn(None, *my_problem[1:], eps=eps, iter_max=n_iter)
        costs_greenkhorn[(n_iter, eps)] = (cost(my_problem[1], transport_plans_greenkhorn[(n_iter, eps)]))
        timing_greenkhorn = %timeit -o -r 2 -n 1 greenkhorn( * my_problem, eps, iter_max=n_iter)
        timings_greenkhorn[(n_iter, eps)] = np.mean(timing_greenkhorn.timings)
        std_timings_greenkhorn[(n_iter, eps)] = np.std(timing_greenkhorn.timings)
        realised_iters_greenkhorn[(n_iter, eps)] = _
        converge_greenkhorn[(n_iter, eps)] = _ < min(n_iter if n_iter is not None else np.inf,
                                                     theoretical_iters_greenkhorn[(n_iter, eps)])

    transport_plans_apdamd[(n_iter, eps)], _ = apdamd(*my_problem, eps=eps, iter_max=n_iter)
    costs_apdamd[(n_iter, eps)] = cost(my_problem[1], transport_plans_apdamd[(n_iter, eps)])
    timing_apdamd = %timeit -o -r 5 -n 1 apdamd( * my_problem, eps, iter_max=n_iter)
    timings_apdamd[(n_iter, eps)] = np.mean(timing_apdamd.timings)
    std_timings_apdamd[(n_iter, eps)] = np.std(timing_apdamd.timings)
    realised_iters_apdamd[(n_iter, eps)] = _
    converge_apdamd[(n_iter, eps)] = _ < min(n_iter if n_iter is not None else np.inf,
                                             theoretical_iters_apdamd[(n_iter, eps)])

np.savez('./run.npz', timings_greenkhorn=timings_greenkhorn, timings_apdamd=timings_apdamd,
         std_timings_greenkhorn=std_timings_greenkhorn, std_timings_apdamd=std_timings_apdamd,
         transport_plans_greenkhorn=transport_plans_greenkhorn, transport_plans_apdamd=transport_plans_apdamd,
         costs_greenkhorn=costs_greenkhorn, costs_apdamd=costs_apdamd, realised_iters_greenkhorn=realised_iters_greenkhorn,
         realised_iters_apdamd=realised_iters_apdamd, theoretical_iters_greenkhorn=theoretical_iters_greenkhorn,
         theoretical_iters_apdamd=theoretical_iters_apdamd, converge_greenkhorn=converge_greenkhorn,
         converge_apdamd=converge_apdamd, epss=epss, my_problem=my_problem)